In [1]:
%pwd

'/media/kirti/Dev/GenAI/E2E_Job_Recommender/notebooks'

In [2]:
import os
os.chdir('../')
%pwd

'/media/kirti/Dev/GenAI/E2E_Job_Recommender'

In [3]:
from src.job_recommender.schema.output_schema import ResumeSummary
from src.job_recommender.models.llm_model import LLMJobAssistant

In [4]:
assistant = LLMJobAssistant()

In [ ]:
summary_prompt_doc = assistant._load_file('prompts/summary_prompt.txt')

In [6]:
summary_prompt_doc

'Summarize the following resume highlighting Skills, Education, Experience. Here is the resume:\n\n{resume_text}\n'

In [7]:
final_prompt = summary_prompt_doc.format(resume_text=sample_resume)
print(final_prompt)

Summarize the following resume highlighting Skills, Education, Experience. Here is the resume:

Kirti Pogra
Email: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra

Objective
----------
Passionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute to innovative projects.

Education
----------
M.S. in Computer Science
XYZ University, 2021

B.Tech in Information Technology
ABC Institute of Technology, 2019

Skills
----------
- Programming: Python, TensorFlow, PyTorch, Scikit-learn, SQL
- Machine Learning: Supervised & Unsupervised Learning, Deep Learning, NLP, Computer Vision
- Data Engineering: Pandas, NumPy, Data Preprocessing, Feature Engineering
- Tools: Docker, Git, MLflow, AWS, Azure
- Communication & Collaboration

Experience
----------
Machine Learning Engineer | T

In [8]:
summary_prompt_template = assistant._get_template()
summary_prompt = summary_prompt_template.invoke({"prompt":final_prompt})

In [9]:
summary_prompt

ChatPromptValue(messages=[HumanMessage(content='Summarize the following resume highlighting Skills, Education, Experience. Here is the resume:\n\nKirti Pogra\nEmail: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra\n\nObjective\n----------\nPassionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute to innovative projects.\n\nEducation\n----------\nM.S. in Computer Science\nXYZ University, 2021\n\nB.Tech in Information Technology\nABC Institute of Technology, 2019\n\nSkills\n----------\n- Programming: Python, TensorFlow, PyTorch, Scikit-learn, SQL\n- Machine Learning: Supervised & Unsupervised Learning, Deep Learning, NLP, Computer Vision\n- Data Engineering: Pandas, NumPy, Data Preprocessing, Feature Engineering\n- Tools: Docker, Git, MLflow, AWS, Azure\n- Communicati

In [10]:
from langchain_core.runnables import RunnableLambda

In [ ]:
format_prompt = prompt['file_content'].format(resume_text=sample_resume)

In [ ]:
def load_prompt(inputs):
    file_path = inputs['file_path']
    kwargs = {k: v for k, v in inputs.items() if k != "file_path"}
    # print(kwargs)

    prompt = assistant._load_file(file_path)
    prompt = prompt.format(**kwargs)
    return {"prompt": prompt}

In [61]:
prompt = RunnableLambda(load_prompt)

In [63]:
prompt_data = prompt.invoke({"file_path":'prompts/summary_prompt.txt',"resume_text": sample_resume})

In [ ]:
prompt_data

{'prompt': 'Summarize the following resume highlighting Skills, Education, Experience. Here is the resume:\n\nKirti Pogra\nEmail: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra\n\nObjective\n----------\nPassionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute to innovative projects.\n\nEducation\n----------\nM.S. in Computer Science\nXYZ University, 2021\n\nB.Tech in Information Technology\nABC Institute of Technology, 2019\n\nSkills\n----------\n- Programming: Python, TensorFlow, PyTorch, Scikit-learn, SQL\n- Machine Learning: Supervised & Unsupervised Learning, Deep Learning, NLP, Computer Vision\n- Data Engineering: Pandas, NumPy, Data Preprocessing, Feature Engineering\n- Tools: Docker, Git, MLflow, AWS, Azure\n- Communication & Collaboration\n\nExperience\n--

In [64]:
summary_prompt_template = assistant._get_template()

In [65]:
summary_prompt_template.invoke(prompt_data)

ChatPromptValue(messages=[HumanMessage(content='Summarize the following resume highlighting Skills, Education, Experience. Here is the resume:\n\nKirti Pogra\nEmail: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra\n\nObjective\n----------\nPassionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute to innovative projects.\n\nEducation\n----------\nM.S. in Computer Science\nXYZ University, 2021\n\nB.Tech in Information Technology\nABC Institute of Technology, 2019\n\nSkills\n----------\n- Programming: Python, TensorFlow, PyTorch, Scikit-learn, SQL\n- Machine Learning: Supervised & Unsupervised Learning, Deep Learning, NLP, Computer Vision\n- Data Engineering: Pandas, NumPy, Data Preprocessing, Feature Engineering\n- Tools: Docker, Git, MLflow, AWS, Azure\n- Communicati

In [66]:
template = assistant._get_template()
model = assistant._get_model()
model = model.with_structured_output(ResumeSummary)


In [56]:
chain = prompt | template | model


In [57]:
chain.get_graph().print_ascii()

    +-------------------+      
    | load_prompt_input |      
    +-------------------+      
              *                
              *                
              *                
       +-------------+         
       | load_prompt |         
       +-------------+         
              *                
              *                
              *                
    +--------------------+     
    | ChatPromptTemplate |     
    +--------------------+     
              *                
              *                
              *                
        +----------+           
        | ChatGroq |           
        +----------+           
              *                
              *                
              *                
   +---------------------+     
   | PydanticToolsParser |     
   +---------------------+     
              *                
              *                
              *                
+---------------------------+  
| Pydant

In [58]:
# input_dict = {
    
# }
result = chain.invoke({"file_path":"prompts/summary_prompt.txt",
                       "resume_text":sample_resume})

# print(result)

{'resume_text': 'Kirti Pogra\nEmail: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra\n\nObjective\n----------\nPassionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute to innovative projects.\n\nEducation\n----------\nM.S. in Computer Science\nXYZ University, 2021\n\nB.Tech in Information Technology\nABC Institute of Technology, 2019\n\nSkills\n----------\n- Programming: Python, TensorFlow, PyTorch, Scikit-learn, SQL\n- Machine Learning: Supervised & Unsupervised Learning, Deep Learning, NLP, Computer Vision\n- Data Engineering: Pandas, NumPy, Data Preprocessing, Feature Engineering\n- Tools: Docker, Git, MLflow, AWS, Azure\n- Communication & Collaboration\n\nExperience\n----------\nMachine Learning Engineer | Tech Innovators Inc. | June 2021 – Present\n- Developed

In [59]:
result

ResumeSummary(name='Kirti Pogra', skills=['Python', 'TensorFlow', 'PyTorch', 'Scikit-learn', 'SQL', 'Docker', 'Git', 'MLflow', 'AWS', 'Azure'], experience=['Machine Learning Engineer, Tech Innovators Inc., June 2021 – Present', 'Data Science Intern, Smart Analytics Ltd., Jan 2020 – May 2021'], education=['M.S. in Computer Science, XYZ University, 2021', 'B.Tech in Information Technology, ABC Institute of Technology, 2019'])

### Trial

In [6]:
from src.job_recommender.chains.llm_chain import JobRecommenderChain
from src.job_recommender.schema.output_schema import ResumeSummary
from src.job_recommender.models.llm_model import LLMJobAssistant


In [4]:
chain = JobRecommenderChain(ResumeSummary)

In [7]:
sample_resume = LLMJobAssistant._load_file('resume.txt')

In [10]:
result=chain.invoke({"file_path":"prompts/summary_prompt.txt",
              "resume_text":sample_resume})

In [ ]:
result.name #type: ignore

'Kirti Pogra'

In [15]:
type(result)

src.job_recommender.schema.output_schema.ResumeSummary